In [1]:
import math
import pandas as pd
import numpy as np
from pprint import pprint

from scipy import constants
from astropy import constants as astro_const
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import pbhs as pbhs
import black_holes as bh

pd.set_option('display.width', 500)

# solar_mass_grams = 1000 * astro_const.M_sun.value

# PBHs

## Quantum evaporation and absorption of energy

Black holes emit thermal radiation at a temperature inversely proportional to their mass. Numerically, this effect is given by

\begin{equation}
T_{bh}=\frac{\hbar{c}^{3}}{8\pi{k_{B}G}M}\simeq \frac{6.2\times{10}^{-8}K}{(M/M_{\odot})}
\end{equation}




In [2]:
solar_mass_units = np.arange(1, 5)

masses_kg = solar_mass_units * astro_const.M_sun.value

# Calculate the Hawking temperature for each mass
hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, astro_const.M_sun.value) for m in masses_kg]

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_solar) from import': solar_mass_units,
    'Mass (kg)': masses_kg,
    'Hawking Temperature (K)': hawking_temperatures
})

# Display the table
print(df)

   Mass (M_solar) from import     Mass (kg)  Hawking Temperature (K)
0                           1  1.988410e+30             6.200000e-08
1                           2  3.976820e+30             3.100000e-08
2                           3  5.965230e+30             2.066667e-08
3                           4  7.953639e+30             1.550000e-08


In [3]:
temperature_critical_mass = pbhs.critical_mass_for_bg_temperature(2.7) / pbhs.solar_mass_grams
dmdt, first, second, _ = pbhs.dm_dt(5 * pbhs.solar_mass_grams, 2.7)

print(f"mass_c({temperature_critical_mass} K): {temperature_critical_mass:.2e}")
print(f"dmdt: {dmdt:.2e}")
print(f" - first: {first:.2e}")
print(f" - second: {second:.2e}")


mass_c(2.2910545872296856e-08 K): 2.29e-08
dmdt: 9.19e-11
 - first: -4.01e-44
 - second: 9.19e-11


In [5]:
background_temperature = 2.7
# solar_mass_units = np.arange(1, 100)
solar_mass_units = np.logspace(start=-20, stop=1, num=50, base=10)


temperature_critical_mass = pbhs.critical_mass_for_bg_temperature(background_temperature)
print(96*"-")
print(f"Planck mass (g): {pbhs.planck_mass_g:.2e} grams")
print(f"critical_mass({background_temperature} K): {temperature_critical_mass:.2e} grams")
print(f"critical_mass({background_temperature} K): {temperature_critical_mass/pbhs.solar_mass_grams:.2e} M_sun")
print(96*"-")

masses_g = solar_mass_units * pbhs.solar_mass_grams
# print("\nMasses(g)")
# pprint(masses_g)

hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, pbhs.solar_mass_grams) for m in masses_g]
# print("\nHawking Temperatures(K)")
# pprint(hawking_temperatures)

terms_tuple = [pbhs.dm_dt(m, background_temperature) for m in masses_g]
# print("\nterms_tuple")
# pprint(terms_tuple)

dm_dt_column, first_term_col, second_term_col, error_diff = zip(*terms_tuple)
# print("\ndm_dt_column")
# pprint(dm_dt_column[0])

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_sun)': solar_mass_units,
    'Mass (g)': masses_g,
    'Hawking temp. (K)': hawking_temperatures,
    # 'dM/dt': dm_dt_column,
    'First term (Hawking Evaporation)': first_term_col,
    'Second term ()': second_term_col,
    'Close?': error_diff
})

output_string = df.to_string(
    # Disable the row index for a cleaner table look
    index=False,
    # Apply a string format to center or left-align the text in each column.
    # We use a formatter dictionary to force a display style.
    # Note: Explicitly adding '|' is tricky, but setting a format helps.
)

# Display the table
print(df.to_markdown(floatfmt='.2e', index=False))

------------------------------------------------------------------------------------------------
Planck mass (g): 2.18e-05 grams
critical_mass(2.7 K): 4.56e+25 grams
critical_mass(2.7 K): 2.29e-08 M_sun
------------------------------------------------------------------------------------------------
|   Mass (M_sun) |   Mass (g) |   Hawking temp. (K) |   First term (Hawking Evaporation) |   Second term () | Close?   |
|---------------:|-----------:|--------------------:|-----------------------------------:|-----------------:|:---------|
|       1.00e-20 |   1.99e+13 |            6.20e+12 |                          -1.00e-02 |         3.68e-52 | False    |
|       2.68e-20 |   5.33e+13 |            2.31e+12 |                          -1.39e-03 |         2.65e-51 | False    |
|       7.20e-20 |   1.43e+14 |            8.61e+11 |                          -1.93e-04 |         1.90e-50 | False    |
|       1.93e-19 |   3.84e+14 |            3.21e+11 |                          -2.69e-05 |     